In [ ]:
from pasteur.kedro.ipython import *

register_kedro()

In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
from pasteur.metadata import Metadata
from pasteur.transform import TableTransformer, Attributes, get_dtype

view = "mimic_tab_admissions"
trn: TableTransformer = catalog.load(f"{view}.trn.table")
table: pd.DataFrame = catalog.load(f"{view}.wrk.idx_table")


In [ ]:
# sensitive
table.head()


In [ ]:
attrs = trn["idx"].get_attributes()


In [ ]:
# from pasteur.synth.math import expand_table, calc_marginal, calc_marginal_1way, AttrSelector, AttrSelectors
# cols, cols_noncommon, domains = expand_table(attrs, table)
from functools import reduce


In [ ]:
from typing import Callable

col_idx = {}
groups = []
heights = []
common = []
domains = []
for i, a in enumerate(attrs.values()):
    for n, c in a.cols.items():
        col_idx[n] = i
        groups.append(i)
        heights.append(c.lvl.height)
        common.append(a.common)
        domains.append([c.get_domain(h) for h in range(c.height)])

n = len(col_idx)
empty_pset = tuple(-1 for _ in range(n))


def add_to_pset(s, x, h):
    s = list(s)
    s[x] = h
    return tuple(s)


In [ ]:
def maximal_parents(V: tuple[int], tau: float, _pgroups=set()) -> list[tuple[int]]:
    if tau < 1:
        return []
    if not V:
        return [empty_pset]
    x = V[0]
    V = V[1:]

    S = []
    U = set()

    for h in range(heights[x]):
        dom = domains[x][h]
        g = groups[x]
        if g in _pgroups:
            dom -= common[x]
            _pgroups_with_x = _pgroups
        else:
            _pgroups_with_x = _pgroups.copy()
            _pgroups_with_x.add(g)

        for z in maximal_parents(V, tau / dom, _pgroups_with_x):
            if z not in U:
                U.add(z)
                S.append(add_to_pset(z, x, h))

    for z in maximal_parents(V, tau, _pgroups):
        if z not in U:
            S.append(z)

    return S


In [ ]:
psets = maximal_parents(tuple(range(n)), 100000000)
len(psets)


400866


In [ ]:
%timeit maximal_parents(tuple(range(n)), 10000)

1.11 s ± 7.88 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%lprun -f maximal_parents maximal_parents(tuple(range(n)), 10000)

Timer unit: 1e-06 s

Total time: 8.71678 s
File: /tmp/ipykernel_2390497/3433675883.py
Function: maximal_parents at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def maximal_parents(V: tuple[int], tau: float, _pgroups=set()) -> list[tuple[int]]:
     2   1246985     552431.0      0.4      6.3      if tau < 1:
     3    307524     119424.0      0.4      1.4          return []
     4    939461     366322.0      0.4      4.2      if not V:
     5    324176     133626.0      0.4      1.5          return [empty_pset]
     6    615285     259700.0      0.4      3.0      x = V[0]
     7    615285     289405.0      0.5      3.3      V = V[1:]
     8                                           
     9    615285     238856.0      0.4      2.7      S = []
    10    615285     275013.0      0.4      3.2      U = set()
    11                                           
    12   1246984     624042.0      0.5      7.2      for h in

In [ ]:
from pasteur.synth.privbayes import greedy_bayes, PrivBayesSynth

In [ ]:
s = PrivBayesSynth(e2=10000)
s.bake(attrs={"table": attrs}, data={"table": table}, ids=None)

WARNING  Considering e2=10000 unbounded, t will be bound to min(n/10, 1e+05)=41947.20 for computational reasons.       privbayes.py:328
>>>>>>>  Finding Nodes: 100%|##########################################################################| 23/23 [00:00<00:00,  1.00it/s]
INFO     Bayesian Network Tree:                                                                                        privbayes.py:559
         (PrivBayes e1=0.30, e2=10000.00, theta=4.00, available t=41947.20)                                                            
         ┌─────────────────────┬─────┬──────────┬─────────────────────────────────────────────────────────┐                            
         │           Attribute │ Dom │ Avail. t │ Parents                                                 │                            
         ├─────────────────────┼─────┼──────────┼─────────────────────────────────────────────────────────┤                            
         │      admittime_year │ 128 │   327.71 

In [ ]:
s.fit(data={"table": table}, ids=None)

WARNING  Considering e2=10000 unbounded, sampling without DP.                                                          privbayes.py:571


In [ ]:
node_names = [n.col for n in s.nodes]
for c, m in zip(node_names, s.marginals):
    print(c, m.shape, m.shape[0]*m.shape[1])

admittime_year (128, 1) 128
insurance (3, 128) 384
marital_status (5, 384) 1920
admission_type (9, 1920) 17280
admission_location (12, 135) 1620
discharge_location (14, 1620) 22680
dischtime_time (48, 210) 10080
admittime_time (48, 210) 10080
admittime_week (52, 210) 10920
dischtime_week (52, 468) 24336
dischtime_day (7, 2704) 18928
admittime_day (7, 4410) 30870
ethnicity (8, 1620) 12960
gender (2, 17280) 34560
language (2, 18720) 37440
dod_year (129, 256) 33024
dod_day (8, 4032) 32256
dod_week (53, 520) 27560
hospital_expire_flag (2, 26460) 52920
deathtime_time (49, 784) 38416
deathtime_day (8, 6720) 53760
deathtime_year (129, 208) 26832
deathtime_week (53, 1344) 71232
dischtime_year (128, 312) 39936


In [ ]:
s.nodes[-1]


Node(
    attr='dischtime',
    col='dischtime_year',
    domain=128,
    p={
        'dod': AttrSelector(common=1, cols={'dod_year': 1, 'dod_week': 1, 'dod_day': 1}),
        'marital_status': AttrSelector(common=1, cols={'marital_status': 0})
    }
)


In [ ]:
attrs["marital_status"].cols["marital_status"].lvl.size

5
